In [56]:
import pandas as pd
import numpy as np

full=pd.merge(pd.read_csv("family_data.csv"),pd.read_csv("sample_submission.csv"))
full['assigned_day']=full['choice_0']
full['assigned_choice']=0
full.head() 

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_day,assigned_choice
0,0,52,38,12,82,33,75,64,76,10,28,4,52,0
1,1,26,4,82,5,11,47,38,6,66,61,4,26,0
2,2,100,54,25,12,27,82,10,89,80,33,3,100,0
3,3,2,95,1,96,32,6,40,31,9,59,2,2,0
4,4,53,1,47,93,26,3,46,16,42,39,4,53,0


In [33]:
def prefCost(people,choice=-1):
    consolation=[[0,0],[50,0],[50,9],[100,9],[200,9],[200,18],[300,18],[300,36],[400,36],[500,36],[500,36+398]]
    return (consolation[choice][0]+consolation[choice][1]*people) 

def accCost(currDay,prevDay=None):
    #For case no. 100, currDay=prevDay
    if prevDay==None: prevDay=currDay
    return (str(currDay)+str(prevDay))

def fullCost(full):
    segregated=full.groupby(by=['assigned_day']).count()['family_id']
    total=0
    for day in range(len(segregated)):
        total+=prefCost(full.iloc[day]['n_people'],full.iloc[day]['assigned_choice'])+

In [55]:
segregated[101]

KeyError: 101